In [4]:
import pandas as pd
import re

In [5]:
df_glovo = pd.read_csv(r'../../data/df_glovo.csv')
df_ta = pd.read_csv(r'../../data/df_ta.csv')

In [6]:
# When I merged both dataframes on the 'Name' column and removed the duplicates, I only had about 200 exact matches. 
# As there might be different formats of restaurnt names (with symbols, numbers, different spacing etc) I made a new column
# With the restaurant name without symbols, spacing etc and then merged the dataframes on that column
df_ta['Name_Clean'] = df_ta['Name'].str.replace(r'[^a-zA-Z]+', '').str.lower()
df_glovo['Name_Clean'] = df_glovo['Name'].str.replace(r'[^a-zA-Z]+', '').str.lower()

C:\Users\Tim_K\AppData\Local\Temp\ipykernel_4568\2134872861.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_ta['Name_Clean'] = df_ta['Name'].str.replace(r'[^a-zA-Z]+', '').str.lower()
C:\Users\Tim_K\AppData\Local\Temp\ipykernel_4568\2134872861.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_glovo['Name_Clean'] = df_glovo['Name'].str.replace(r'[^a-zA-Z]+', '').str.lower()


In [7]:
# Merging the dataframes on the 'name_clean' column
merged_df = pd.merge(df_ta, df_glovo, on='Name_Clean', how='inner')


#Now there are a few columns that have duplicat names, which are chains (McDonalds, Burger king, Pizza circus etc) we only keep 1 of them.
merged_df = merged_df.drop_duplicates(subset='Name_Clean', keep='first')

# Removing all columns we don't need
merged_df.drop(columns='Unnamed: 0_x',inplace=True)
merged_df.drop(columns='Unnamed: 0_y',inplace=True)
merged_df.drop(columns='Name_y',inplace=True)
merged_df.drop(columns='Name_Clean',inplace=True)


# Renaming columns so it's clear what is what 
merged_df = merged_df.rename(columns={
    'Name_x' :  'Name',
    'Cuisine_Type': 'Cuisine Style TA',
    'Cuisine Style': 'Cuisine Style Glovo',
    'Rating': 'Rating TA',
    'Review_Count': 'Number of Reviews TA',
    'Cuisine Style': 'Cuisine Style Glovo',
    'Rating Glovo': 'Rating Glovo',
    'Number of Reviews': 'Number of Reviews Glovo'
})
# Ordering the columns 
merged_df = merged_df[['Name', 'Rating TA', 'Rating Glovo', 'Cuisine Style TA', 'Cuisine Style Glovo', 'Number of Reviews TA', 'Number of Reviews Glovo']]
merged_df


,Name,Rating TA,Rating Glovo,Cuisine Style TA,Cuisine Style Glovo,Number of Reviews TA,Number of Reviews Glovo
0,Bella Venezia,5.0,98%,['Italian'],['Pizza'],102,207
143,makis,4.5,97%,"['Sushi', 'Asian']",['Japanese'],78,500+
236,Casa Bangla,4.5,88%,"['Indian', 'Bangladeshi']",['Indian'],5,65
407,Indian Herbs,5.0,95%,"['Indian', 'Healthy']",['Indian'],157,237
408,Sumac & Mambo,5.0,89%,"['Barbecue', 'Mediterranean']",['Mediterranean'],542,18
...,...,...,...,...,...,...,...
932,Pita House,1.5,92%,['Spanish'],"['Arabic', 'Pizza']",874,119
934,Wok Chow Mein,4.5,91%,"['Chinese', 'Asian']",['Asian'],5,58
935,Entre Tapes Meridiana,4.5,82%,['Mediterranean'],"['Chicken', 'Mediterranean']",5,18
936,4 Pokes,5.0,97%,['Hawaiian'],"['Poke', 'Vegan']",23,500+


In [8]:
#As in the glovo dataframe the rating is expressed in a percentage (1-100%), we transform the rating from tripadvisor (now 0.0 - 5.0) also to %.
def rating_to_percentage(rating):
    if pd.isnull(rating):
        return rating
    return int(rating * 20)

merged_df['Rating TA'] = merged_df['Rating TA'].apply(rating_to_percentage)
merged_df['Rating TA'] = merged_df['Rating TA'].apply(lambda x: str(int(x)) + '%' if not pd.isna(x) else x)

merged_df

,Name,Rating TA,Rating Glovo,Cuisine Style TA,Cuisine Style Glovo,Number of Reviews TA,Number of Reviews Glovo
0,Bella Venezia,100%,98%,['Italian'],['Pizza'],102,207
143,makis,90%,97%,"['Sushi', 'Asian']",['Japanese'],78,500+
236,Casa Bangla,90%,88%,"['Indian', 'Bangladeshi']",['Indian'],5,65
407,Indian Herbs,100%,95%,"['Indian', 'Healthy']",['Indian'],157,237
408,Sumac & Mambo,100%,89%,"['Barbecue', 'Mediterranean']",['Mediterranean'],542,18
...,...,...,...,...,...,...,...
932,Pita House,30%,92%,['Spanish'],"['Arabic', 'Pizza']",874,119
934,Wok Chow Mein,90%,91%,"['Chinese', 'Asian']",['Asian'],5,58
935,Entre Tapes Meridiana,90%,82%,['Mediterranean'],"['Chicken', 'Mediterranean']",5,18
936,4 Pokes,100%,97%,['Hawaiian'],"['Poke', 'Vegan']",23,500+


In [ ]:
merged_df.to_csv(r'../../data/df_glovo_ta.csv')